In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommender/coupon/train.csv')
df_coupon = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommender/coupon/coupon_item_mapping.csv')
df_item = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommender/coupon/item_data.csv')

In [ ]:
df_item['category'].unique()

array(['Grocery', 'Miscellaneous', 'Bakery', 'Pharmaceutical',
       'Packaged Meat', 'Seafood', 'Natural Products',
       'Dairy, Juices & Snacks', 'Prepared Food', 'Skin & Hair Care',
       'Meat', 'Travel', 'Flowers & Plants', 'Fuel', 'Salads', 'Alcohol',
       'Garden', 'Restauarant', 'Vegetables (cut)'], dtype=object)

In [ ]:
df_coupon.head()

,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [ ]:
df_item.head()

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [ ]:
coupon = pd.merge(df_coupon, df_item, on='item_id')
coupon.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,6,37,56,Local,Grocery
2,22,37,56,Local,Grocery
3,31,37,56,Local,Grocery
4,107,75,56,Local,Grocery


In [ ]:
df = pd.merge(df_train, coupon, on='coupon_id')
df.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,item_id,brand,brand_type,category
0,1,13,27,1053,0,1404,1636,Established,Grocery
1,1,13,27,1053,0,1418,1105,Established,Grocery
2,1,13,27,1053,0,1449,1636,Established,Grocery
3,1,13,27,1053,0,1515,1105,Established,Grocery
4,1,13,27,1053,0,1532,1636,Established,Grocery


In [ ]:
columns = ['id', 'campaign_id', 'brand_type']
df.drop(columns, axis=1, inplace=True)
df.head()

,coupon_id,customer_id,redemption_status,item_id,brand,category
0,27,1053,0,1404,1636,Grocery
1,27,1053,0,1418,1105,Grocery
2,27,1053,0,1449,1636,Grocery
3,27,1053,0,1515,1105,Grocery
4,27,1053,0,1532,1636,Grocery


In [ ]:
df['redemption_status'].unique()

array([0, 1])

In [ ]:
combine_coupon_rating = df.dropna(axis = 0, subset = ['category'])
coupon_ratingCount = (combine_coupon_rating.groupby(by = ['category'])['redemption_status'].count().
     reset_index().
     rename(columns = {'redemption_status': 'totalRatingCount'})
     [['category', 'totalRatingCount']]
    )
coupon_ratingCount.head()

,category,totalRatingCount
0,Bakery,7117
1,"Dairy, Juices & Snacks",146362
2,Flowers & Plants,133874
3,Garden,19463
4,Grocery,2694248


In [ ]:

rating_with_totalRatingCount = combine_coupon_rating.merge(coupon_ratingCount, left_on = 'category', right_on = 'category', how = 'left')
rating_with_totalRatingCount.head()

,coupon_id,customer_id,redemption_status,item_id,brand,category,totalRatingCount
0,27,1053,0,1404,1636,Grocery,2694248
1,27,1053,0,1418,1105,Grocery,2694248
2,27,1053,0,1449,1636,Grocery,2694248
3,27,1053,0,1515,1105,Grocery,2694248
4,27,1053,0,1532,1636,Grocery,2694248


In [ ]:
rating_with_totalRatingCount['totalRatingCount'].mean()

1645321.9146895958

In [ ]:
popularity_threshold = 100000
rating_popular_coupon= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_coupon.head()

,coupon_id,customer_id,redemption_status,item_id,brand,category,totalRatingCount
0,27,1053,0,1404,1636,Grocery,2694248
1,27,1053,0,1418,1105,Grocery,2694248
2,27,1053,0,1449,1636,Grocery,2694248
3,27,1053,0,1515,1105,Grocery,2694248
4,27,1053,0,1532,1636,Grocery,2694248


In [ ]:
coupon_features_df=rating_popular_coupon.pivot_table(index='category',columns='customer_id',values='redemption_status').fillna(0)
coupon_features_df.head()

customer_id,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27,28,30,31,32,33,34,36,37,38,39,40,41,42,43,44,...,1537,1538,1539,1540,1541,1542,1543,1544,1545,1546,1547,1549,1550,1551,1552,1554,1555,1556,1557,1558,1559,1560,1562,1563,1564,1565,1566,1567,1568,1570,1571,1573,1574,1575,1576,1578,1579,1580,1581,1582
category,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Dairy, Juices & Snacks",0.000000,0.0,0.0,0.0,0.0,0.000000,0.959410,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.625604,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.003221,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.982019,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000
Flowers & Plants,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000
Grocery,0.005288,0.0,0.0,0.0,0.0,0.005814,0.825959,0.0,0.0,0.0,0.012265,0.0,0.0,0.155078,0.0,0.0,0.0,0.0,0.0,0.035014,0.0,0.0,0.0,0.0,0.0,0.060201,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.051932,0.071157,0.0,0.0,0.0,0.006649,0.418095,0.0,0.0,0.082816,0.0,0.0,0.022310,0.0,0.380030,0.0,0.0,0.0,0.31694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036687,0.839939,0.0,0.0,0.008329,0.0,0.0,0.0,0.02939
Meat,0.000000,0.0,0.0,0.0,0.0,0.000000,0.002458,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.727273,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000
Natural Products,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.323529,0.0,0.0,0.000000,0.0,0.0,0.916667,0.0,0.003127,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.993191,0.0,0.0,0.000000,0.0,0.0,0.0,0.00000


In [ ]:
from scipy.sparse import csr_matrix

coupon_features_df_matrix = csr_matrix(coupon_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(coupon_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
coupon_features_df.loc['Flowers & Plants', :]

customer_id
1       0.0
3       0.0
4       0.0
5       0.0
6       0.0
       ... 
1578    0.0
1579    0.0
1580    0.0
1581    0.0
1582    0.0
Name: Flowers & Plants, Length: 1428, dtype: float64

In [ ]:
reqd_coupon = 'Grocery'
distances, indices = model_knn.kneighbors(coupon_features_df.loc[reqd_coupon, :].values.reshape(1, -1), n_neighbors = 6)

In [ ]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(reqd_coupon))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, coupon_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Grocery:

1: Dairy, Juices & Snacks, with distance of 0.22386080946798526:
2: Packaged Meat, with distance of 0.4521703377240355:
3: Seafood, with distance of 0.4559595246959556:
4: Natural Products, with distance of 0.5271885620806676:
5: Meat, with distance of 0.615995840146512:
